## Load configurations

In [1]:
from cfgs.content_classification_v1 import get_cfg

cfgs = get_cfg()
print('\n', cfgs, '\n')


 CLASSES: [{'Genre': {0: 'Comedy', 1: 'Drama', 2: 'Action', 3: 'Documentary', 4: 'Crime', 5: 'Animation', 6: 'Horror', 7: 'Adventure'}, 'Rating': {0: 'value'}, 'Year': {0: 'value'}}]
HYPERPARAMS:
  BATCH_SIZE_TR: 32
  BATCH_SIZE_VAL: 32
  EPOCHS: 25
  LR: 0.0001
  TEST_SPLIT: 0.8
  VAL_SPLIT: 0.8
IMAGE:
  RESOLUTION: (200, 150, 3)
MODEL:
  BACKBONE: InceptionV3
  STRUCTURE: [{0: {'type': 'Genre', 'outNeurons': 8, 'outActivation': 'sigmoid', 'loss': 'categorical_crossentropy', 'weight': 1, 'metric': 'accuracy'}, 1: {'type': 'Rating', 'outNeurons': 1, 'outActivation': 'linear', 'loss': 'mse', 'weight': 1, 'metric': 'mae'}, 2: {'type': 'Year', 'outNeurons': 1, 'outActivation': 'linear', 'loss': 'mse', 'weight': 1, 'metric': 'mae'}}] 



In [2]:
dataset_dict = cfgs.CLASSES[0]
dataset_dict['Genre_alias'] = dict((g, i) for i, g in dataset_dict['Genre'].items())

## Load data and initialize generator

In [3]:
import pandas as pd
from data_processing.data_generator import DataGenerator

In [4]:
df = pd.read_pickle("/home/robotics/content-classification/data_processing/balanced_data.pkl")

In [5]:
data_generator = DataGenerator(df, dataset_dict, cfgs.HYPERPARAMS.TEST_SPLIT, cfgs.HYPERPARAMS.VAL_SPLIT, cfgs.IMAGE.RESOLUTION)
data_generator.df.head()

,Index,Name,Year,Genre,Rating,Poster_path
0,tt0110901,Project Shadowchaser II,1995,2,4.0,/home/robotics/Documents/data/Posters/1995/tt0...
1,tt0365265,Ginger Snaps Back: The Beginning,2004,1,5.9,/home/robotics/Documents/data/Posters/2004/tt0...
2,tt0244992,Barrela: Escola de Crimes,1990,1,6.5,/home/robotics/Documents/data/Posters/1990/tt0...
3,tt1648188,Sharunas Bartas: An Army of One,2010,3,6.5,/home/robotics/Documents/data/Posters/2010/tt1...
4,tt1935737,Aquatic Language,2002,5,6.3,/home/robotics/Documents/data/Posters/2002/tt1...


In [6]:
train_idx, valid_idx, test_idx = data_generator.split_dataset() 

## Build and compile model

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from model.architecture import MultiOutputModel

In [8]:
structure = cfgs.MODEL.STRUCTURE[0]
content_model = MultiOutputModel(*cfgs.IMAGE.RESOLUTION, structure)
content_model.build_model("InceptionV3")
# keras.utils.plot_model(content_model.model, show_shapes=True)

In [9]:
opt = Adam(lr=cfgs.HYPERPARAMS.LR, decay=cfgs.HYPERPARAMS.LR/cfgs.HYPERPARAMS.EPOCHS)
content_model.compile_model(opt)
content_model.model.summary()

Model: "content_classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 99, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 99, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 99, 74, 32)   0           batch_normalization[0][0]        
_____________________________________________________________________________

## Train Model

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.backend import clear_session
import tensorflow as tf

In [11]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

# GPU check - CUDA 11.1 with cuDNN 8.05 on 460.39 - GTX 1060 6GB
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

clear_session()

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
tr_generator = data_generator.generate_images(train_idx, cfgs.HYPERPARAMS.BATCH_SIZE_TR, True)
val_generator = data_generator.generate_images(valid_idx, cfgs.HYPERPARAMS.BATCH_SIZE_VAL, True)

In [13]:
import datetime

%load_ext tensorboard
log_dir = log_dir = "./logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = [keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)]
# keras.callbacks.ModelCheckpoint('./checkpoints', monitor='val_loss', verbose=0, 
#                                              save_best_only=False, save_weights_only=False, 
#                                              mode='auto', save_freq='epoch')

In [14]:
history = content_model.model.fit(tr_generator, 
                    steps_per_epoch=len(train_idx)//cfgs.HYPERPARAMS.BATCH_SIZE_TR,
                    epochs=cfgs.HYPERPARAMS.EPOCHS,
                    callbacks=callbacks,
                    validation_data=val_generator,
                    validation_steps=len(valid_idx)//cfgs.HYPERPARAMS.BATCH_SIZE_VAL)

Epoch 1/25
 55/320 [====>.........................] - ETA: 47s - loss: 5.1816 - Genre_loss: 2.3369 - Rating_loss: 1.6004 - Year_loss: 1.2443 - Genre_accuracy: 0.1532 - Rating_mae: 0.8221 - Year_mae: 0.7821
* * * * * Image could not be loaded - skipping * * * * *

320/320 [==============================] - 75s 214ms/step - loss: 3.0767 - Genre_loss: 2.1064 - Rating_loss: 0.5404 - Year_loss: 0.4300 - Genre_accuracy: 0.2024 - Rating_mae: 0.4444 - Year_mae: 0.4185 - val_loss: 1.9447 - val_Genre_loss: 1.8853 - val_Rating_loss: 0.0339 - val_Year_loss: 0.0255 - val_Genre_accuracy: 0.2793 - val_Rating_mae: 0.1466 - val_Year_mae: 0.1265
INFO:tensorflow:Assets written to: ./checkpoints/assets
Epoch 2/25
 55/320 [====>.........................] - ETA: 49s - loss: 1.9604 - Genre_loss: 1.8339 - Rating_loss: 0.0707 - Year_loss: 0.0557 - Genre_accuracy: 0.3105 - Rating_mae: 0.2057 - Year_mae: 0.1858
* * * * * Image could not be loaded - skipping * * * * *

320/320 [==============================] - 9

KeyboardInterrupt: 

In [15]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 18689), started 0:36:13 ago. (Use '!kill 18689' to kill it.)